In [1]:
import time as t

In [2]:
start_time = t.time()

In [3]:
start_time

1650114829.0359037

In [4]:
from pyspark.sql import functions as sparkf
from pyspark.sql.types import *

In [5]:
import pandas as pd

In [6]:
pd.set_option('display.max_columns', None)

In [7]:
pd.set_option('display.max_rows', None)

# 1. Identifying Duplicate Rows

In [8]:
! hdfs dfs -ls -h /rawzone/example/LoanStats_web.csv

Picked up JAVA_TOOL_OPTIONS: -Dhttps.protocols=TLSv1.2
-rw-r--r--   2 hadoopuser supergroup      1.1 G 2022-04-16 08:53 /rawzone/example/LoanStats_web.csv


In [9]:
raw_df = spark.read.format('csv').option('header','true').option('mode','DROPMALFORMED')\
.option('inferSchema','true')\
.load('hdfs://hadoop-master:9000/rawzone/example/LoanStats_web.csv')

In [10]:
raw_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- member_id: string (nullable = true)
 |-- loan_amnt: integer (nullable = true)
 |-- funded_amnt: integer (nullable = true)
 |-- funded_amnt_inv: double (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: string (nullable = true)
 |-- installment: double (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- emp_title: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: double (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- issue_d: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- pymnt_plan: string (nullable = true)
 |-- url: string (nullable = true)
 |-- desc: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- title: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: strin

In [11]:
raw_df.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
id,0,None,None,None,None
member_id,0,None,None,None,None
loan_amnt,1432439,15370.392945179516,9646.028077240393,1000,40000
funded_amnt,1432439,15370.373398099326,9646.029406782209,1000,40000
funded_amnt_inv,1432439,15365.89239055986,9644.784657109356,725.0,40000.0
term,1432439,None,None,36 months,60 months
int_rate,1432439,None,None,5.31%,30.99%
installment,1432439,456.68919254501196,281.7140757681437,19.4,1715.42
grade,1432439,None,None,A,G


#### นับจำนวน Attributes

In [12]:
len(raw_df.columns)

144

In [13]:
raw_df.count()

1432466

In [14]:
selectedAttr_df = raw_df.select("loan_amnt","term","int_rate","installment","grade","emp_length",\
                           "home_ownership","annual_inc","verification_status","loan_status",\
                           "purpose","addr_state","dti","delinq_2yrs","earliest_cr_line",\
                           "inq_last_6mths","open_acc","pub_rec","revol_bal","revol_util","total_acc",\
                           "last_credit_pull_d")

In [15]:
selectedAttr_df.printSchema()

root
 |-- loan_amnt: integer (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: string (nullable = true)
 |-- installment: double (nullable = true)
 |-- grade: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: double (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: string (nullable = true)
 |-- delinq_2yrs: string (nullable = true)
 |-- earliest_cr_line: string (nullable = true)
 |-- inq_last_6mths: double (nullable = true)
 |-- open_acc: integer (nullable = true)
 |-- pub_rec: integer (nullable = true)
 |-- revol_bal: integer (nullable = true)
 |-- revol_util: string (nullable = true)
 |-- total_acc: integer (nullable = true)
 |-- last_credit_pull_d: string (nullable = true)



In [16]:
from pyspark.sql.window import Window

from pyspark.sql.functions import row_number

checkdupWindow = Window.partitionBy("loan_amnt","term","int_rate","installment","grade","emp_length",\
                           "home_ownership","annual_inc","verification_status","loan_status",\
                           "purpose","addr_state","dti","delinq_2yrs","earliest_cr_line",\
                           "inq_last_6mths","open_acc","pub_rec","revol_bal","revol_util","total_acc",\
                           "last_credit_pull_d").orderBy('loan_amnt')

checkdup_df = selectedAttr_df.withColumn('occur_id',row_number().over(checkdupWindow))

In [17]:
checkdup_df.groupBy('occur_id').count().orderBy('occur_id').show()

+--------+-------+
|occur_id|  count|
+--------+-------+
|       1|1142104|
|       2|  96780|
|       3|  96780|
|       4|  96780|
|       5|      1|
|       6|      1|
|       7|      1|
|       8|      1|
|       9|      1|
|      10|      1|
|      11|      1|
|      12|      1|
|      13|      1|
|      14|      1|
|      15|      1|
|      16|      1|
|      17|      1|
|      18|      1|
|      19|      1|
|      20|      1|
+--------+-------+
only showing top 20 rows



In [18]:
checkdup_df.groupBy('occur_id').count().orderBy('occur_id').agg(sparkf.sum('count')).show()

+----------+
|sum(count)|
+----------+
|   1432466|
+----------+



In [19]:
checkdup_df.count()

1432466

In [20]:
checkdup_df.filter(sparkf.col('occur_id') <= 1).select("loan_amnt","term","int_rate","installment","grade","emp_length",\
                           "home_ownership","annual_inc","verification_status","loan_status",\
                           "purpose","addr_state","dti","delinq_2yrs","earliest_cr_line",\
                           "inq_last_6mths","open_acc","pub_rec","revol_bal","revol_util","total_acc",\
                           "last_credit_pull_d",'occur_id')\
.limit(100).toPandas().transpose()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
loan_amnt,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1075,1200,1200,1200,1200,1200,1200,1200,1200,1200,1200,1200,1200,1250,1275,1300,1300,1400,1400,1450,1500,1500,1500,1500,1500,1500,1500,1500,1500,1500,1500,1500,1600,1600,1600,1600,1600,1600,1600,1650,1650,1675,1675,1700,1800,1800,1800,1800,1800,1900,1900,2000,2000,2000,2000,2000,2000,2000,2000,2000,2000,2000,2000
term,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months
int_rate,6.67%,7.02%,7.39%,7.56%,8.24%,9.49%,9.80%,10.08%,10.99%,11.49%,11.49%,11.49%,12.62%,12.73%,12.74%,12.99%,13.49%,13.99%,14.07%,14.47%,14.49%,14.99%,15.31%,16.99%,17.97%,17.99%,17.99%,18.06%,19.99%,21.85%,22.90%,23.13%,23.40%,23.87%,24.85%,26.31%,30.65%,19.03%,7.84%,9.49%,11.39%,11.44%,12.73%,13.56%,13.99%,14.47%,14.47%,15.99%,16.99%,25.11%,13.59%,9.92%,13.49%,14.99%,7.99%,21.49%,11.49%,7.02%,9.49%,9.93%,11.39%,11.47%,11.80%,12.13%,12.79%,13.67%,20.89%,22.35%,22.90%,7.84%,10.90%,11.55%,16.14%,17.97%,20.00%,25.11%,5.31%,18.99%,10.72%,17.99%,12.79%,10.49%,11.06%,12.62%,12.74%,14.47%,14.47%,23.40%,5.32%,6.11%,6.67%,7.56%,7.84%,8.24%,8.39%,8.81%,9.16%,10.07%,10.75%,11.05%
installment,30.73,30.89,31.06,31.14,31.45,32.03,32.18,32.31,32.74,32.98,32.98,32.98,33.52,33.57,33.57,33.69,33.94,34.18,34.22,34.41,34.42,34.67,34.82,35.65,36.14,36.15,36.15,36.19,37.16,38.12,38.66,38.78,38.92,39.17,39.69,40.46,42.81,39.43,37.52,38.44,39.51,39.54,40.28,40.76,41.01,41.29,41.29,42.19,42.78,47.79,42.48,41.1,44.11,45.06,43.87,53.1,47.81,46.33,48.05,48.36,49.39,49.45,49.68,49.92,50.39,51.03,56.43,57.56,57.99,50.03,52.31,52.8,56.37,57.82,59.47,63.71,49.69,60.48,54.62,60.55,57.11,58.5,58.99,60.33,60.43,61.94,65.38,73.95,60.23,60.95,61.46,62.27,62.53,62.9,63.04,63.43,63.75,64.61,65.25,65.53
grade,A,A,A,A,B,B,B,B,B,B,B,B,C,B,C,C,C,C,C,C,C,C,C,D,D,D,D,D,D,D,E,F,E,E,E,E,F,D,A,B,B,B,B,C,C,C,C,C,D,F,C,B,C,C,A,D,B,A,B,B,B,B,B,B,C,C,D,E,E,A,B,B,C,D,D,F,A,D,B,D,C,B,B,C,C,C,C,E,A,A,A,A,A,B,B,A,B,B,B,B
emp_length,10+ years,< 1 year,4 years,7 years,5 years,n/a,n/a,2 years,n/a,10+ years,10+ years,3 years,6 years,2 years,1 year,n/a,10+ years,2 years,10+ years,10+ years,3 years,n/a,5 years,2 years,10+ years,10+ years,3 years,4 years,n/a,2 years,5 years,2 years,< 1 year,3 years,10+ years,5 years,8 years,7 years,2 years,10+ years,n/a,10+ years,10+ years,n/a,2 years,7 years,n/a,8 years,n/a,1 year,10+ years,10+ years,10+ years,1 year,10+ years,1 year,10+ years,2 years,7 years,2 years,2 years,10+ years,< 1 year,n/a,2 years,10+ years,10+ years,9 years,2 years,5 years,1 year,2 years,10+ years,2 years,10+ years,4 years,3 years,< 1 year,4 years,10+ years,5 years,4 years,10+ years,3 years,1 year,10+ 

In [21]:
checkdup_df.filter(sparkf.col('occur_id') > 1).select("loan_amnt","term","int_rate","installment","grade","emp_length",\
                           "home_ownership","annual_inc","verification_status","loan_status",\
                           "purpose","addr_state","dti","delinq_2yrs","earliest_cr_line",\
                           "inq_last_6mths","open_acc","pub_rec","revol_bal","revol_util","total_acc",\
                           "last_credit_pull_d",'occur_id')\
.limit(100).toPandas().transpose()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
loan_amnt,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1200,1200,1200,1200,1200,1200,1200,1200,1200,1800,1800,1800,2000,2000,2000,2000,2000,2000,2000,2000,2000,2000,2000,2000,2100,2100,2100,2100,2100,2100,2200,2200,2200,2200,2200,2200,2400,2400,2400,2500,2500,2500,2800,2800,2800,2800,2800,2800,3000,3000,3000,3000,3000,3000,3000,3000,3000,3000,3000,3000,3000,3000,3000,3000,3000,3000,3000,3000,3000,3000,3000,3000,3025,3025,3025,3200,3200,3200,3500
term,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months,36 months
int_rate,11.49%,11.49%,11.49%,12.74%,12.74%,12.74%,13.49%,13.49%,13.49%,13.99%,13.99%,13.99%,14.99%,14.99%,14.99%,16.99%,16.99%,16.99%,17.99%,17.99%,17.99%,11.39%,11.39%,11.39%,11.44%,11.44%,11.44%,15.99%,15.99%,15.99%,12.74%,12.74%,12.74%,8.24%,8.24%,8.24%,11.39%,11.39%,11.39%,11.39%,11.39%,11.39%,14.99%,14.99%,14.99%,6.99%,6.99%,6.99%,11.49%,11.49%,11.49%,5.32%,5.32%,5.32%,8.24%,8.24%,8.24%,14.99%,14.99%,14.99%,13.99%,13.99%,13.99%,10.49%,10.49%,10.49%,14.99%,14.99%,14.99%,8.24%,8.24%,8.24%,11.44%,11.44%,11.44%,11.44%,11.44%,11.44%,11.49%,11.49%,11.49%,11.49%,11.49%,11.49%,13.49%,13.49%,13.49%,13.49%,13.49%,13.49%,23.99%,23.99%,23.99%,11.49%,11.49%,11.49%,11.44%,11.44%,11.44%,10.49%
installment,32.98,32.98,32.98,33.57,33.57,33.57,33.94,33.94,33.94,34.18,34.18,34.18,34.67,34.67,34.67,35.65,35.65,35.65,36.15,36.15,36.15,39.51,39.51,39.51,39.54,39.54,39.54,42.19,42.19,42.19,60.43,60.43,60.43,62.9,62.9,62.9,65.85,65.85,65.85,65.85,65.85,65.85,69.33,69.33,69.33,64.84,64.84,64.84,69.24,69.24,69.24,66.26,66.26,66.26,69.19,69.19,69.19,83.19,83.19,83.19,85.44,85.44,85.44,91,91,91,97.05,97.05,97.05,94.35,94.35,94.35,98.85,98.85,98.85,98.85,98.85,98.85,98.92,98.92,98.92,98.92,98.92,98.92,101.8,101.8,101.8,101.8,101.8,101.8,117.69,117.69,117.69,99.74,99.74,99.74,105.44,105.44,105.44,113.75
grade,B,B,B,C,C,C,C,C,C,C,C,C,C,C,C,D,D,D,D,D,D,B,B,B,B,B,B,C,C,C,C,C,C,B,B,B,B,B,B,B,B,B,C,C,C,A,A,A,B,B,B,A,A,A,B,B,B,C,C,C,C,C,C,B,B,B,C,C,C,B,B,B,B,B,B,B,B,B,B,B,B,B,B,B,C,C,C,C,C,C,E,E,E,B,B,B,B,B,B,B
emp_length,10+ years,10+ years,10+ years,1 year,1 year,1 year,10+ years,10+ years,10+ years,2 years,2 years,2 years,n/a,n/a,n/a,2 years,2 years,2 years,10+ years,10+ years,10+ years,n/a,n/a,n/a,10+ years,10+ years,10+ years,8 years,8 years,8 years,1 year,1 year,1 year,10+ years,10+ years,10+ years,3 years,3 years,3 years,8 years,8 years,8 years,10+ years,10+ years,10+ years,4 years,4 years,4 years,5 years,5 years,5 years,6 years,6 years,6 years,< 1 year,< 1 year,< 1 year,5 years,5 years,5 years,5 years,5 years,5 years,3 years,3 years,3 years,6 years,6 years,6 years,7 years,7 years,7 years,6 years,6 years,6 years,< 1 year,< 1 year,< 1 year,10+ years,10+ years,10+ years,< 1 year,<

# 2. Deduplicate Rows

In [22]:
nodup_df = checkdup_df.filter(sparkf.col('occur_id') <= 1)

In [23]:
nodup_df.count()

1142104

In [24]:
nodup_df.distinct().count()

1142104

In [25]:
nodup_df.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
loan_amnt,1142103,15500.57278108892,9704.19713217745,1000,40000
term,1142103,None,None,36 months,60 months
int_rate,1142103,None,None,5.31%,30.99%
installment,1142103,458.17351101433053,281.2427618244707,19.4,1715.42
grade,1142103,None,None,A,G
emp_length,1142103,None,None,1 year,n/a
home_ownership,1142103,None,None,ANY,RENT
annual_inc,1142103,80595.69649127088,104098.99683451139,0.0,6.1E7
verification_status,1142103,None,None,Not Verified,Verified


# 3. Validator for identifying invalid data

In [26]:
validator = [('int_rate','valid_int_rate','\d+.\d+%'),\
            ('dti','valid_dti','\d+')]

In [27]:
for v in range(len(validator)):
    print('\n\n---------- Apply Validator: ',validator[v][2],' to: ', validator[v][0],' ----------\n\n')
    nodup_df.withColumn(validator[v][1]\
                    ,sparkf.when((sparkf.col(validator[v][0]).rlike(validator[v][2])),'valid').otherwise('invalid'))\
.select(validator[v][0],validator[v][1])\
.groupBy(validator[v][1])\
.count().orderBy(validator[v][1],ascending=False).show(100)

    nodup_df.withColumn(validator[v][1]\
                    ,sparkf.when((sparkf.col(validator[v][0]).rlike(validator[v][2])),'valid').otherwise('invalid'))\
.select(validator[v][0],validator[v][1])\
.filter(sparkf.col(validator[v][1])=='valid')\
.groupBy(validator[v][0],validator[v][1]).count().show(100)

    nodup_df.withColumn(validator[v][1]\
                    ,sparkf.when((sparkf.col(validator[v][0]).rlike(validator[v][2])),'valid').otherwise('invalid'))\
.select(validator[v][0],validator[v][1])\
.filter(sparkf.col(validator[v][1])=='invalid')\
.groupBy(validator[v][0],validator[v][1]).count().show(100)



---------- Apply Validator:  \d+.\d+%  to:  int_rate  ----------


+--------------+-------+
|valid_int_rate|  count|
+--------------+-------+
|         valid|1142103|
|       invalid|      1|
+--------------+-------+

+--------+--------------+-----+
|int_rate|valid_int_rate|count|
+--------+--------------+-----+
|   8.59%|         valid| 3904|
|  13.49%|         valid|20302|
|   9.92%|         valid| 6562|
|   6.08%|         valid| 2968|
|   6.67%|         valid| 8751|
|   9.80%|         valid| 2224|
|  26.24%|         valid| 1369|
|   9.49%|         valid| 7083|
|  26.14%|         valid|  134|
|  24.11%|         valid|  801|
|  28.80%|         valid|  625|
|  25.49%|         valid| 1649|
|  15.31%|         valid| 8131|
|  30.49%|         valid|  400|
|  11.44%|         valid|11006|
|  24.49%|         valid| 1410|
|  14.52%|         valid| 3913|
|  29.96%|         valid|  149|
|  14.03%|         valid| 3963|
|  28.34%|         valid|  195|
|  21.99%|         valid|    2|
|  12.62%|  

# 4. Transform invalid data to be valid one

In [28]:
valid_df = nodup_df.withColumn('int_rate',sparkf.regexp_replace('int_rate','%','').cast(DoubleType()))\
.filter(sparkf.col('int_rate').isNotNull())\
.withColumn('dti',sparkf.regexp_extract('dti','(\d+)',1).cast(StringType()))\
.filter(sparkf.col('dti').isNotNull())\
.filter(sparkf.col('dti')!='')

In [29]:
validator = [('int_rate','valid_int_rate','\d+.\d+'),\
            ('dti','valid_dti','\d+')]

In [30]:
for v in range(len(validator)):
    print('\n\n---------- Apply Validator: ',validator[v][2],' to: ', validator[v][0],' ----------\n\n')
    valid_df.withColumn(validator[v][1]\
                    ,sparkf.when((sparkf.col(validator[v][0]).rlike(validator[v][2])),'valid').otherwise('invalid'))\
.select(validator[v][0],validator[v][1])\
.groupBy(validator[v][1])\
.count().orderBy(validator[v][1],ascending=False).show(100)

    valid_df.withColumn(validator[v][1]\
                    ,sparkf.when((sparkf.col(validator[v][0]).rlike(validator[v][2])),'valid').otherwise('invalid'))\
.select(validator[v][0],validator[v][1])\
.filter(sparkf.col(validator[v][1])=='valid')\
.groupBy(validator[v][0],validator[v][1]).count().show(100)

    valid_df.withColumn(validator[v][1]\
                    ,sparkf.when((sparkf.col(validator[v][0]).rlike(validator[v][2])),'valid').otherwise('invalid'))\
.select(validator[v][0],validator[v][1])\
.filter(sparkf.col(validator[v][1])=='invalid')\
.groupBy(validator[v][0],validator[v][1]).count().show(100)



---------- Apply Validator:  \d+.\d+  to:  int_rate  ----------


+--------------+-------+
|valid_int_rate|  count|
+--------------+-------+
|         valid|1140617|
+--------------+-------+

+--------+--------------+-----+
|int_rate|valid_int_rate|count|
+--------+--------------+-----+
|    8.59|         valid| 3904|
|   19.48|         valid|  637|
|   12.88|         valid| 1816|
|     9.8|         valid| 2224|
|   25.81|         valid| 1193|
|   29.69|         valid|  581|
|   13.18|         valid|   34|
|   10.75|         valid|11683|
|    6.83|         valid| 3903|
|   10.08|         valid| 9537|
|   14.85|         valid| 1438|
|    7.56|         valid| 8414|
|   15.49|         valid| 3820|
|    28.9|         valid|   52|
|   11.55|         valid|11910|
|    9.43|         valid| 6775|
|    18.2|         valid|    3|
|    7.34|         valid| 6164|
|   11.22|         valid|   27|
|   25.11|         valid|  220|
|   11.99|         valid|18667|
|    7.39|         valid| 8758|
|   11